## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

import sys
sys.path.insert(0, '../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,62.60,88,40,13.80,scattered clouds
1,1,Castro,BR,-24.7911,-50.0119,70.29,91,46,7.31,scattered clouds
2,2,Bac Lieu,VN,9.2850,105.7244,72.81,86,13,10.87,few clouds
3,3,Avarua,CK,-21.2078,-159.7750,84.20,74,41,10.36,scattered clouds
4,4,Tabas,IR,33.5959,56.9244,37.40,17,0,4.61,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Castro,BR,-24.7911,-50.0119,70.29,91,46,7.31,scattered clouds
2,2,Bac Lieu,VN,9.2850,105.7244,72.81,86,13,10.87,few clouds
5,5,Port Elizabeth,ZA,-33.9180,25.5701,71.60,73,40,24.16,scattered clouds
90,90,Kidal,ML,18.4411,1.4078,70.25,20,57,13.04,broken clouds
91,91,Sao Filipe,CV,14.8961,-24.4956,73.35,65,89,16.33,overcast clouds
92,92,Huarmey,PE,-10.0681,-78.1522,74.37,72,78,8.08,broken clouds
112,112,Dingle,PH,10.9995,122.6711,74.30,93,23,6.67,heavy intensity rain
135,135,Uchiza,PE,-8.4592,-76.4633,74.03,85,99,1.61,light rain
137,137,Sechura,PE,-5.5569,-80.8222,70.41,85,100,4.74,light rain
140,140,Boende,CD,-0.2167,20.8667,74.86,75,54,3.20,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
city_data_df.count()

City_ID                698
City                   698
Country                691
Lat                    698
Lng                    698
Max Temp               698
Humidity               698
Cloudiness             698
Wind Speed             698
Current Description    698
dtype: int64

In [6]:
city_data_df.isnull().sum()

City_ID                0
City                   0
Country                7
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,62.60,88,40,13.80,scattered clouds
1,1,Castro,BR,-24.7911,-50.0119,70.29,91,46,7.31,scattered clouds
2,2,Bac Lieu,VN,9.2850,105.7244,72.81,86,13,10.87,few clouds
3,3,Avarua,CK,-21.2078,-159.7750,84.20,74,41,10.36,scattered clouds
4,4,Tabas,IR,33.5959,56.9244,37.40,17,0,4.61,clear sky
...,...,...,...,...,...,...,...,...,...,...
693,693,Ponta Delgada,PT,37.7333,-25.6667,61.00,88,40,17.27,scattered clouds
694,694,Kristiansund,NO,63.1115,7.7320,23.00,74,0,3.44,clear sky
695,695,Uray,RU,60.1247,64.7758,3.36,94,100,4.70,overcast clouds
696,696,Kamaishi,JP,39.2667,141.8833,44.80,79,38,16.49,scattered clouds


In [8]:
clean_df.count()

City_ID                691
City                   691
Country                691
Lat                    691
Lng                    691
Max Temp               691
Humidity               691
Cloudiness             691
Wind Speed             691
Current Description    691
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ribeira Grande,PT,62.60,scattered clouds,38.5167,-28.7000,
1,Castro,BR,70.29,scattered clouds,-24.7911,-50.0119,
2,Bac Lieu,VN,72.81,few clouds,9.2850,105.7244,
3,Avarua,CK,84.20,scattered clouds,-21.2078,-159.7750,
4,Tabas,IR,37.40,clear sky,33.5959,56.9244,
5,Port Elizabeth,ZA,71.60,scattered clouds,-33.9180,25.5701,
6,Santa Cruz De La Palma,ES,64.40,broken clouds,28.6835,-17.7642,
7,Tuatapere,NZ,62.53,few clouds,-46.1333,167.6833,
8,Punta Arenas,CL,55.40,clear sky,-53.1500,-70.9167,
9,Qaanaaq,GL,9.32,overcast clouds,77.4840,-69.3632,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
    #except Exception as e:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ribeira Grande,PT,62.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
1,Castro,BR,70.29,scattered clouds,-24.7911,-50.0119,CHACARA BAILLY
2,Bac Lieu,VN,72.81,few clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
3,Avarua,CK,84.20,scattered clouds,-21.2078,-159.7750,Paradise Inn
4,Tabas,IR,37.40,clear sky,33.5959,56.9244,شرکت معماری مهراز شایگان آراد
...,...,...,...,...,...,...,...
652,Ankang,CN,40.86,overcast clouds,32.6800,109.0172,安琪酒店
662,Taverny,FR,32.00,clear sky,49.0254,2.2169,Hôtel Restaurant Campanile Taverny
663,Sur,OM,66.09,clear sky,22.5667,59.5289,Sur Plaza Hotel
669,Perivolion,GR,51.01,scattered clouds,39.4206,20.0147,Santa Maria


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))